In [19]:
import fennol
import yaml
import jax
import glob
from copy import deepcopy

In [20]:
with open("ani_ensemble.yaml") as f:
    yaml_data = yaml.safe_load(f)
print(yaml_data)

{'energy_unit': 'Ha', 'energy_terms': ['energy'], 'cutoff': 5.1, 'preprocessing': {'graph_filter': {'cutoff': 3.5, 'graph_key': 'graph_angle', 'parent_graph': 'graph'}, 'graph_angular_extension': {'graph_key': 'graph_angle'}, 'species_indexer': {'output_key': 'species_index'}}, 'modules': {'embedding': {'angle_sections': 4, 'angular_dist_divisions': 8, 'angular_eta': 12.5, 'angular_start': 0.8, 'embedding_key': 'ani_aev', 'graph_angle_key': 'graph_angle', 'graph_key': 'graph', 'module_name': 'ANI_AEV', 'radial_dist_divisions': 16, 'radial_eta': 19.7, 'radial_start': 0.8, 'species_order': ['H', 'C', 'N', 'O', 'S', 'F', 'Cl'], 'zeta': 14.1}, 'energy0': {'activation': 'partial(celu, alpha=0.1)', 'hidden_neurons': {'C': [224, 192, 160], 'Cl': [160, 128, 96], 'F': [160, 128, 96], 'H': [256, 192, 160], 'N': [192, 160, 128], 'O': [192, 160, 128], 'S': [160, 128, 96]}, 'input_key': 'ani_aev', 'module_name': 'SPECIES_INDEX_NET', 'output_dim': 1, 'species_index_key': 'species_index', 'use_bias':

In [21]:
model = fennol.FENNIX(**yaml_data, rng_key=jax.random.PRNGKey(0))
print(model.variables["params"].keys())

dict_keys(['energy0', 'energy1', 'energy2', 'energy3', 'energy4', 'energy5', 'energy6', 'energy7', 'self_energy'])


In [22]:
variables = deepcopy(model.variables)

In [25]:
model_files = sorted(glob.glob("ANI2X/ani2x_model*.fnx"))
for i, model_file in enumerate(model_files):
    mi = fennol.load(model_file)
    if i==0:
      variables["params"]["self_energy"] = mi.variables["params"]["self_energy"]  
    variables["params"][f"energy{i}"]= mi.variables["params"]["energy"]



In [27]:
print(variables["params"].keys())

dict_keys(['energy0', 'energy1', 'energy2', 'energy3', 'energy4', 'energy5', 'energy6', 'energy7', 'self_energy'])


In [26]:
model.variables = variables
model.save("ANI2X/ani2x_ensemble.fnx")